In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os.path
import base64
import email
from tqdm import trange
from bs4 import BeautifulSoup
import lxml

In [2]:
# Define the SCOPES. If modifying it, delete the token.pickle file.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

In [45]:
def getEmails():
    # Variable creds will store the user access token.
    # If no valid token found, we will create one.
    creds = None
    
    monthMap = { "Jan" : 1, "Feb" : 2, "Mar" : 3, "Apr" : 4, "May" : 5, "June" : 6,
       "July" : 7, "Aug" : 8, "Sept" : 9, "Oct" : 10, "Nov" : 11, "Dec" : 12 }
  
    # The file token.pickle contains the user access token.
    # Check if it exists
    if os.path.exists('token.pickle'):
  
        # Read the token from the file and store it in the variable creds
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
  
    # If credentials are not available or are invalid, ask the user to log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
  
        # Save the access token in token.pickle file for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
  
    # Connect to the Gmail API
    service = build('gmail', 'v1', credentials = creds)
  
    # request a list of all the messages
    result = service.users().messages().list(userId='me').execute()
  
    # We can also pass maxResults to get any number of emails. Like this:
    # result = service.users().messages().list(maxResults=200, userId='me').execute()
    messages = result.get('messages')
  
    # messages is a list of dictionaries where each dictionary contains a message id.
  
    # iterate through all the messages
    for i in trange(1):
        # Get the message from its id
        txt = service.users().messages().get(userId = 'me', id = messages[i]['id']).execute()
  
        # Get value of 'payload' from dictionary 'txt'
        payload = txt['payload']
        headers = payload['headers']

        # Look for Subject and Sender Email in the headers
        for d in headers:
            if d['name'] == 'Subject':
                subject = d['value']
            if d['name'] == 'Date':
                date = d['value'][6:17]
                date = date.replace(" ", ",")
                
                temp = date.split(",")
                day = temp[0]
                month = str(monthMap[temp[1]])
                year = temp[2]
                
                date = year + "_" + month + "_" + day

        # The Body of the message is in Encrypted format. So, we have to decode it.
        # Get the data and decode it with base 64 decoder.
        body = payload.get('body')
        data = body['data']
        data = data.replace("-","+").replace("_","/")
        decoded_data = base64.b64decode(data)

        # Now, the data obtained is in lxml. So, we will parse 
        # it with BeautifulSoup library
        soup = BeautifulSoup(decoded_data , "lxml")
        result = soup.body()

        # Printing the subject, sender's email and message
        print("Date: ", date)
        print("Subject: ", subject)
        print("Message: ", result)
        print('\n')

In [46]:
getEmails()

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.47it/s]

Date:  2022_3_8
Subject:  【女王寵愛禮】$150購物金送給您！美腿塑形神器$1xx起 / 迷你復古音響$3xx / 電動睫毛卷$2xx 點我逛>>
Message:  [<div align="center">
<table border="0" cellpadding="0" cellspacing="0" id="table1">
<tr>
<td background="https://mkt.ruten.com.tw/edm/6826/edm/images/edm_01.jpg" height="92"><table border="0" cellpadding="0" cellspacing="0" height="100%" id="table14" width="100%">
<tr>
<td valign="top" width="18%"><table border="0" cellpadding="0" cellspacing="0" height="100%" id="table15" width="100%">
<tr>
<td height="58">　</td>
</tr>
<tr>
<td align="right" height="26"><p>
<font color="#000000" face="Verdana" style="font-size: 12px;">2022/03/08</font></p></td>
</tr>
</table></td>
<td>　</td>
<td valign="top"><table border="0" cellpadding="0" cellspacing="0" height="100%" id="table16" width="100%">
<tr>
<td align="right" height="58">　</td>
</tr>
<tr>
<td align="right"><font color="#000000" face="arial" size="2">
<p align="right" style="margin-right: 13px; margin-top:8px"><a href="http://class.ruten.com.tw/